In [1]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score

In [2]:
def cross_val_alpha(X, X_test, X_train, y_test, y_train, reg_type='lasso' ,plot_graph=True):
    # cross validation
    l_min = 0.001
    l_max = 0.8
    l_num = 30
    lambdas = np.linspace(l_min,l_max, l_num)
    
    train_r_squared = np.zeros(l_num)
    test_r_squared = np.zeros(l_num)

    pred_num = X.shape[1]
    coeff_a = np.zeros((l_num, pred_num))

    for ind, i in enumerate(lambdas):    
        if reg_type == 'lasso':
            reg = linear_model.Lasso(alpha = i)
            reg.fit(X_train, y_train)
            results = cross_val_score(reg, X, y, cv=5, scoring="r2")
            train_r_squared[ind] = reg.score(X_train, y_train)
            test_r_squared[ind] = reg.score(X_test, y_test)
        elif reg_type == 'ridge':
            reg = linear_model.Ridge(alpha = i)
            reg.fit(X_train, y_train)
            results = cross_val_score(reg, X, y, cv=5, scoring="r2")
            train_r_squared[ind] = reg.score(X_train, y_train)
            test_r_squared[ind] = reg.score(X_test, y_test)
        elif reg_type == 'elasticnet':
            reg = linear_model.ElasticNet(alpha = i)
            reg.fit(X_train, y_train)
            results = cross_val_score(reg, X, y, cv=5, scoring="r2")
            train_r_squared[ind] = reg.score(X_train, y_train)
            test_r_squared[ind] = reg.score(X_test, y_test)

    # #optional plot
    if plot_graph == True:

        plt.figure(figsize=(10, 5))
        plt.plot(train_r_squared, 'bo-', label=r'$R^2$ Training set', color="darkblue", alpha=0.6, linewidth=3)
        plt.plot(test_r_squared, 'bo-', label=r'$R^2$ Test set', color="darkred", alpha=0.6, linewidth=3)
        plt.xlabel('Lamda value'); plt.ylabel(r'$R^2$')
        plt.xlim(0, 19)
        plt.title(r'Evaluate 5-fold cv with different lamdas')
        plt.legend(loc='best')
        plt.grid()

    df_lam = pd.DataFrame(test_r_squared*100, columns=['R_squared'])
    df_lam['lambda'] = (lambdas)

    # returns the index of the row where column has maximum value.
    return df_lam.loc[df_lam['R_squared'].idxmax()]

In [3]:
# output metrics, and plot predicted vs original results
def metrics_plot(y1, y2, plot_graph=True):
    #calculate metrics
    mae = mean_absolute_error(y1, y2)
    mse = mean_squared_error(y1, y2)
    rmse = math.sqrt(mse)
    r2 = r2_score(y1, y2)
    
    #output metrics
    print('MSE:  {}'.format(round(mse, 3)))
    print('RMSE: {}'.format(round(rmse, 3)))
    print('MAE:  {}'.format(round(mae, 3)))
    print('R2:   {}'.format(round(r2, 3)))

    #plot graph
    if plot_graph == True:
        plt.figure(figsize=(8,8))
        plt.plot(y1, y2, 'co')
        plt.plot([100,1500], [100,1500], color='black', linewidth=2.0, linestyle='-')
    
        plt.xlabel('Measured')
        plt.ylabel('Predicted')
        plt.grid(True)
        plt.xlim((100,1500))
        plt.ylim((100,1500))
        
        my_x_ticks = np.arange(100,1600,200)
        my_y_ticks = np.arange(100, 1600, 200)
        plt.xticks(my_x_ticks)
        plt.yticks(my_y_ticks)
        plt.axis('scaled')
    
        plt.show()

    return mse, rmse, mae, r2

In [4]:
# read date from csv file
data = pd.read_csv('fatigue_data.csv', index_col='Sl. No.')

# delete extra columns
xx = data.drop(data.columns[17:20], axis=1)
xx.head()

,NT,THT,THt2,THQCr,CT,Ct3,DT,Dt4,QmT,TT,Tt5,TCr,C,Ni,Cr,Mo,Fatigue
Sl. No.,,,,,,,,,,,,,,,,,
1,885,30,0,0,30,0.0,30.0,0.0,30,30,0,0.0,0.26,0.01,0.02,0.0,232
2,885,30,0,0,30,0.0,30.0,0.0,30,30,0,0.0,0.25,0.08,0.12,0.0,235
3,885,30,0,0,30,0.0,30.0,0.0,30,30,0,0.0,0.26,0.02,0.03,0.0,235
4,885,30,0,0,30,0.0,30.0,0.0,30,30,0,0.0,0.26,0.01,0.02,0.0,241
5,885,30,0,0,30,0.0,30.0,0.0,30,30,0,0.0,0.22,0.01,0.02,0.0,225


In [5]:
# set X as columns C, Ni, Cr and Mo (No.16-19)
X = data.drop(data.columns[16:20], axis=1) 
# set y as 'Fatigue' column (No.17)
y = data['Fatigue']

X.shape, y.shape

((437, 16), (437,))

In [7]:
# splitting data into training set and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

# calculate learning rates for lasso, ridge and elasticnet regressions
alpha_las = cross_val_alpha(X, X_test, X_train, y_test, y_train, plot_graph=False)
alpha_rig = cross_val_alpha(X, X_test, X_train, y_test, y_train, reg_type='ridge', plot_graph=False)
alpha_el = cross_val_alpha(X, X_test, X_train, y_test, y_train, reg_type='elasticnet', plot_graph=False)

alpha_las[1], alpha_rig[1], alpha_el[1]

(0.11120689655172414, 0.08365517241379311, 0.001)

In [8]:
# simple linear regression model
lr = linear_model.LinearRegression().fit(X_train, y_train)
y_pred = lr.predict(X_test)

# lasso regression model
lr_las = linear_model.Lasso(0.1112).fit(X_train, y_train)
y_pred_las = lr_las.predict(X_test)

# ridge regression model
lr_rig = linear_model.Ridge(0.0837).fit(X_train, y_train)
y_pred_rig = lr_rig.predict(X_test)

# elasticnet regression model
lr_el = linear_model.Ridge(0.001).fit(X_train, y_train)
y_pred_el = lr_el.predict(X_test)

In [11]:
mse_lr, rmse_lr, mae_lr, r2_lr = metrics_plot(y_test, y_pred, plot_graph=False)

MSE:  1243.081
RMSE: 35.257
MAE:  24.817
R2:   0.97


In [12]:
mse_las, rmse_las, mae_las, r2_las = metrics_plot(y_test, y_pred_las, plot_graph=False)

MSE:  1288.933
RMSE: 35.902
MAE:  25.218
R2:   0.969


In [13]:
mse_rig, rmse_rig, mae_rig, r2_rig = metrics_plot(y_test, y_pred_rig, plot_graph=False)

MSE:  1218.391
RMSE: 34.905
MAE:  24.428
R2:   0.971


In [14]:
mse_el, rmse_el, mae_el, r2_el = metrics_plot(y_test, y_pred_el, plot_graph=False)

MSE:  1242.184
RMSE: 35.245
MAE:  24.806
R2:   0.97
